In [ ]:
import pandas as pd
import numpy as np
import requests
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
from datetime import datetime

In [13]:
# Mengambil data historis pengguna dari API
def fetch_purchase_history(api_url):
    response = requests.get(api_url)
    if response.status_code == 200:
        data = response.json()
        return pd.DataFrame(data)
    else:
        raise Exception("Failed to fetch purchase history")

# Memuat data pertandingan dari CSV
def load_match_data(csv_path):
    return pd.read_csv(csv_path)

# URL API dan path CSV
api_url = "https://raw.githubusercontent.com/alvinfebrianto/BolaTix-API/refs/heads/main/bolatix.json"
csv_path = "/content/DATASET - data liga 1.csv"  # Ganti dengan path sebenarnya

# Mengambil data
purchase_history = fetch_purchase_history(api_url)
matches = load_match_data(csv_path)

print("Data Historis Pengguna:")
print(purchase_history.head())

print("\nData Pertandingan:")
print(matches.head())

Data Historis Pengguna:
   userId userName favoriteTeam idMatch                          match  \
0       5    Rizky       Persib                                          
1       4     Kiki  Bali United       1            Persib vs PSBS Biak   
2       3   Jeffry    Persebaya       2  Madura United vs Malut United   
3       2     Aryo     Arema FC       3         PSM Makassar vs PERSIS   
4       1  Aransha      Persija       4       Persija vs Barito Putera   

        date   time                        stadium  
0                                                   
1   9/8/2024  20:00       Stadion Si Jalak Harupat  
2  10/8/2024  16:30      Stadion Gelora Bangkalan   
3  10/8/2024  16:30                Stadion Batakan  
4  10/8/2024  20:00  Jakarta International Stadium  

Data Pertandingan:
   ID Match             Home           Away                           Match  \
0         1          Persib       PSBS Biak             Persib vs PSBS Biak   
1         2  Madura United     Malu

In [ ]:
# Konversi tipe data idMatch menjadi int, menangani nilai kosong
purchase_history['idMatch'] = pd.to_numeric(purchase_history['idMatch'], errors='coerce').fillna(0).astype(int)

# Verifikasi setelah konversi
print("Tipe data idMatch di purchase_history setelah konversi:", purchase_history['idMatch'].dtype)

Tipe data idMatch di purchase_history setelah konversi: int64


In [ ]:
# Memastikan konsistensi tipe data
# Ubah ID Match di matches menjadi integer (jika belum)
matches['ID Match'] = matches['ID Match'].astype(int)

In [ ]:
print("Tipe data idMatch di purchase_history:", purchase_history['idMatch'].dtype)
print("Tipe data ID Match di matches:", matches['ID Match'].dtype) #

Tipe data idMatch di purchase_history: int64
Tipe data ID Match di matches: int64


In [ ]:
# Pastikan kolom Tanggal adalah datetime
matches['Tanggal '] = pd.to_datetime(matches['Tanggal '], format='%d/%m/%Y', errors='coerce')

# Isi nilai NaT (NaN untuk datetime) dengan tanggal default atau info lainnya
matches['Tanggal '] = matches['Tanggal '].fillna('Tanggal tidak tersedia')

In [ ]:
# Fungsi utama untuk merekomendasikan
def recommend(user_id, purchase_history, matches, N=10):
    # Filter data pengguna berdasarkan user_id
    user_data = purchase_history[purchase_history['userId'] == user_id]

    if user_data.empty:
        # Jika user_id tidak ada di API
        print("Maaf, ID Anda tidak ada di API.")
        return pd.DataFrame()

    # Jika user sudah memiliki tim kesayangan dari data API
    favorite_teams = user_data['favoriteTeam'].dropna().unique().tolist()

    # Rekomendasi berdasarkan tim kesayangan
    recommendations = recommend_for_favorite_teams(matches, favorite_teams, N)

    return recommendations

In [ ]:
# Rekomendasi untuk pengguna berdasarkan tim kesayangan
def recommend_for_favorite_teams(matches, favorite_teams, N=10):
    # Filter pertandingan yang melibatkan tim favorit (case insensitive)
    favorite_matches = matches[matches['Home'].str.lower().isin([team.lower() for team in favorite_teams]) | matches['Away'].str.lower().isin([team.lower() for team in favorite_teams])].copy()

    # Tampilkan jumlah pertandingan yang ditemukan setelah filter tim favorit
    print(f"Jumlah pertandingan yang melibatkan tim favorit ({favorite_teams}): {len(favorite_matches)}")

    # Pastikan kolom Tanggal adalah datetime
    favorite_matches['Tanggal '] = pd.to_datetime(favorite_matches['Tanggal '], errors='coerce')

    # Menghapus pertandingan yang tidak memiliki tanggal valid
    favorite_matches = favorite_matches.dropna(subset=['Tanggal '])

    # Filter pertandingan yang tanggalnya lebih besar atau sama dengan hari ini
    today = datetime.now()
    favorite_matches = favorite_matches[favorite_matches['Tanggal '] >= today]

    # Tampilkan jumlah pertandingan setelah memfilter tanggal valid
    print(f"Jumlah pertandingan setelah memfilter tanggal valid: {len(favorite_matches)}")

    # Urutkan berdasarkan tanggal untuk memastikan relevansi (tanggal terdekat terlebih dahulu)
    favorite_matches = favorite_matches.sort_values(by='Tanggal ', ascending=True)

    # Ubah format tanggal menjadi tanggal-bulan-tahun
    favorite_matches['Tanggal '] = favorite_matches['Tanggal '].dt.strftime('%d-%m-%Y')

    # Jika terlalu banyak, pilih yang paling baru atau sesuai kriteria lain
    recommendations = favorite_matches.head(N)
    return recommendations

In [ ]:
# Contoh penggunaan
user_id = 3
recommendations = recommend(user_id, purchase_history, matches, N=10)
print("\nRekomendasi:")
if not recommendations.empty:
    print(recommendations[['ID Match', 'Home', 'Away', 'Tanggal ']].to_string(index=False))
else:
    print("Tidak ada rekomendasi yang relevan.")

Jumlah pertandingan yang melibatkan tim favorit (['Persebaya']): 29
Jumlah pertandingan setelah memfilter tanggal valid: 11

Rekomendasi:
 ID Match           Home         Away   Tanggal 
      105  Madura United    Persebaya 02-12-2024
      112      Persebaya     Arema FC 07-12-2024
      120      Persebaya       Persik 11-12-2024
      128  Semen Padang     Persebaya 15-12-2024
      137      Persebaya    Borneo FC 20-12-2024
      150    Bali United    Persebaya 28-12-2024
      157     PSS Sleman    Persebaya 11-01-2025
      163      Persebaya Malut United 17-01-2025
      175 Barito putera     Persebaya 25-01-2025
      182      Persebaya      Persita 31-01-2025
